In [19]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader


In [20]:
class FashionMNISTDataset(Dataset):
    def __init__(self, csv_file, transform = None):
        data = pd.read_csv(csv_file)
        self.X = np.array(data.iloc[:, 1:]).reshape(-1, 28, 28, 1).astype(float)
        
        self.Y = np.array(data.iloc[:, 0])

        del data;
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        item = self.X[idx]
        label = self.Y[idx]
        
        if self.transform:
            item = self.transform(item)
            
        return (item, label)

In [21]:
class ToTensor(object):
    def __call__(self, sample):
        sample = sample.transpose((2, 0, 1))
        return torch.from_numpy(sample)

In [22]:
num_epochs = 10
learning_rate = 0.001
batch_size = 100

In [23]:
train_dataset = FashionMNISTDataset(csv_file = 
                                    '/media/user/D:/Image Recognition QSTP/fashion-mnist_train.csv',
                                   transform = transforms.ToTensor())
test_dataset = FashionMNISTDataset(csv_file = 
                                    '/media/user/D:/Image Recognition QSTP/fashion-mnist_test.csv',
                                  transform = transforms.ToTensor())

In [24]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, 
                                         batch_size = batch_size,
                                         shuffle = False)

In [25]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
                    nn.Conv2d(1, 16, kernel_size = 5, padding = 2),
                    nn.BatchNorm2d(16),
                    nn.ReLU(),
                    nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                    nn.Conv2d(16, 32, kernel_size = 5, padding = 2),
                    nn.BatchNorm2d(32),
                    nn.ReLU(),
                    nn.MaxPool2d(2))
        self.fc = nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [26]:
model = CNN()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [27]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = Variable(images.float())
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
         
        if (i+1) % 100 == 0:
           print('Epoch : %d/%d, Iter : %d/%d, Loss: %.4f'
                %(epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, loss.data[0])) 

Epoch : 1/10, Iter : 100/600, Loss: 0.4722
Epoch : 1/10, Iter : 200/600, Loss: 0.4948
Epoch : 1/10, Iter : 300/600, Loss: 0.3432
Epoch : 1/10, Iter : 400/600, Loss: 0.3491
Epoch : 1/10, Iter : 500/600, Loss: 0.3519
Epoch : 1/10, Iter : 600/600, Loss: 0.2915
Epoch : 2/10, Iter : 100/600, Loss: 0.2126
Epoch : 2/10, Iter : 200/600, Loss: 0.3497
Epoch : 2/10, Iter : 300/600, Loss: 0.3446
Epoch : 2/10, Iter : 400/600, Loss: 0.3069
Epoch : 2/10, Iter : 500/600, Loss: 0.2992
Epoch : 2/10, Iter : 600/600, Loss: 0.2538
Epoch : 3/10, Iter : 100/600, Loss: 0.1316
Epoch : 3/10, Iter : 200/600, Loss: 0.2031
Epoch : 3/10, Iter : 300/600, Loss: 0.3357
Epoch : 3/10, Iter : 400/600, Loss: 0.1383
Epoch : 3/10, Iter : 500/600, Loss: 0.1741
Epoch : 3/10, Iter : 600/600, Loss: 0.1769
Epoch : 4/10, Iter : 100/600, Loss: 0.2084
Epoch : 4/10, Iter : 200/600, Loss: 0.1961
Epoch : 4/10, Iter : 300/600, Loss: 0.2086
Epoch : 4/10, Iter : 400/600, Loss: 0.1379
Epoch : 4/10, Iter : 500/600, Loss: 0.1382
Epoch : 4/1

In [28]:
model.eval()
correct = 0
total = 0

for images, labels in test_loader:
    
    images = Variable(images.float())
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (labels == predicted).sum()
    
print('Test Accuracy of the model: %.4f %%' %(100 * correct / total)) 

Test Accuracy of the model: 91.0900 %
